In [1]:
import sqlite3 as sq1
import pandas as pd

In [2]:
conn =sq1.connect("RSV.db")

try:

    
    conn.execute("""
    DROP TABLE IF EXISTS sequence_epitope
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS genebank
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS major_clade
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS subclade
    """)

    conn.execute("""
    CREATE TABLE IF NOT EXISTS major_clade (subtype TEXT, clade TEXT);
    """)
    
    ## 1 -> M relationship encoded with an identifier on joint table
    conn.execute("""
    CREATE TABLE IF NOT EXISTS subclade (genotype TEXT, nucleotide TEXT, subclade INT,
                    FOREIGN KEY ('genotype')
                    REFERENCES major_clade('clade')
                    ON DELETE CASCADE
                    ON UPDATE CASCADE);
    """)


    
    ## M -> M need a seperated table
    conn.execute("""
    CREATE TABLE IF NOT EXISTS sequence_epitope (id TEXT, nucleotide TEXT, amino_acid TEXT, binding_score INT, 
    PRIMARY KEY ('id'));  
    """)
    
    conn.execute("""
    CREATE TABLE IF NOT EXISTS genebank (acession TEXT, country TEXT, year INT,
                    FOREIGN KEY('acession') 
                    REFERENCES sequence_epitope ('id')
                    ON DELETE CASCADE
                    ON UPDATE CASCADE);  
    """)
    
    
    conn.executemany("""
    INSERT INTO major_clade VALUES (?,?)""",[('A','ON1'),
                                               ('A','ON1'),
                                               ('A','GA2'),
                                               ('B','BA'),
                                               ('B','GB3')])

    
    conn.executemany("""
    INSERT INTO subclade VALUES (?,?,?)""",[('ON1','ATGGACTCTTACT',1),
                                           ('ON1','CTGGACGGTTTCC',2),
                                           ('GA2','GCTTTCGGGATAC',3),
                                           ('BA','ATGGACTCTTACT',2),
                                           ('GB3','GCTTTCGGGATAC',1)])
    
    
    
    conn.executemany("""
    INSERT INTO sequence_epitope VALUES (?, ?, ?,?)""",[('AY911262','ATGGACTCTTACT','AKSWTTNWWT',1),
                                           ('FJ614813','CTGGACGGTTTCC','AKSWTTNWWT' ,3),
                                           ('FJ948820','GCTTTCGGGATAC','TTNSSABKYY',0),
                                           ('JF920046','ATGGACTCTTACT','VMMTTABKYS',4),
                                           ('JF920047','GCTTTCGGGATAC','TTSSMKAYTT',1)])
    
    conn.executemany("""
    INSERT INTO genebank VALUES (?, ?, ?)""",[('AY911262','USA',2007),
                                           ('FJ614813','USA',2009),
                                           ('FJ948820','Netherland',2010),
                                           ('JF920046','USA',2012),
                                           ('JF920047','USA',2009)])
    
    cursor1 = conn.execute("select * from sequence_epitope")
    df1 = pd.DataFrame(cursor1.fetchall(),columns = [ 'id','nucleotide','amino_acid','binding_score'] )              
    print(df1)
    
    cursor2 = conn.execute('''select major_clade.subtype, major_clade.clade, subclade.subclade from
                           major_clade join subclade
                           on major_clade.clade = subclade.genotype''')
                           
    
    df2 = pd.DataFrame(cursor2.fetchall(),columns = [ 'subtype','clade','subclade'] )              
    print(df2)
    
    cursor3 = conn.execute('''select sequence_epitope.id, sequence_epitope.nucleotide, genebank.country ,genebank.year from
                          sequence_epitope join genebank
                           on sequence_epitope.id = genebank.acession
                           where country = 'USA' ''')
    df3 = pd.DataFrame(cursor3.fetchall(),columns = [ 'id','seq','country','year'] )              
    print(df3)
    
    
    cursor4 = conn.execute('''select sequence_epitope.id, sequence_epitope.nucleotide, subclade.genotype from 
                           sequence_epitope join subclade
                           on sequence_epitope.nucleotide = subclade.nucleotide
                           order by sequence_epitope.id asc''')
    df4 = pd.DataFrame(cursor4.fetchall(),columns = [ 'id','nucleotide','genotype'] )              
    print(df4)
    
    cursor5 = conn.execute('''update subclade set genotype = 'GA2' where genotype = 'ON1';''')
    
    cursor5 = conn.execute('''select major_clade.subtype, major_clade.clade, subclade.subclade from
                           major_clade join subclade
                           on major_clade.clade = subclade.genotype''')
    df5 = pd.DataFrame(cursor5.fetchall(),columns = [ 'subtype','clade','genotype'] )              
    print(df5)   
    


                    
                           
    ## force database to update                       
    conn.commit();
    
    
                           
                           
except sq1.Error as e:
    print("There was error:" + str(e))   
    
finally: 
    print("this excuted")
    conn.close()                  
 

         id     nucleotide  amino_acid  binding_score
0  AY911262  ATGGACTCTTACT  AKSWTTNWWT              1
1  FJ614813  CTGGACGGTTTCC  AKSWTTNWWT              3
2  FJ948820  GCTTTCGGGATAC  TTNSSABKYY              0
3  JF920046  ATGGACTCTTACT  VMMTTABKYS              4
4  JF920047  GCTTTCGGGATAC  TTSSMKAYTT              1
  subtype clade  subclade
0       A   ON1         1
1       A   ON1         2
2       A   ON1         1
3       A   ON1         2
4       A   GA2         3
5       B    BA         2
6       B   GB3         1
         id            seq country  year
0  AY911262  ATGGACTCTTACT     USA  2007
1  FJ614813  CTGGACGGTTTCC     USA  2009
2  JF920046  ATGGACTCTTACT     USA  2012
3  JF920047  GCTTTCGGGATAC     USA  2009
         id     nucleotide genotype
0  AY911262  ATGGACTCTTACT       BA
1  AY911262  ATGGACTCTTACT      ON1
2  FJ614813  CTGGACGGTTTCC      ON1
3  FJ948820  GCTTTCGGGATAC      GA2
4  FJ948820  GCTTTCGGGATAC      GB3
5  JF920046  ATGGACTCTTACT       BA
6  JF920046